In [50]:
# Web Scrapping

In [51]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [52]:
# headers for requests
URL = 'https://www.amazon.in/s?k=playstation&crid=L285YXF77263&sprefix=playstati%2Caps%2C695&ref=nb_sb_ss_ts-doa-p_2_9'
HEADERS = ({'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

In [53]:
webpage = requests.get(URL,headers=HEADERS)

In [54]:
webpage

<Response [503]>

In [55]:
webpage = requests.get(URL,headers=HEADERS)
webpage

<Response [503]>

In [56]:
import time

max_retries = 10
retry_delay = 5  # seconds

for attempt in range(max_retries):
    webpage = requests.get(URL, headers=HEADERS)
    
    if webpage.status_code == 200:
        print("Request successful!")
        break
    elif webpage.status_code == 503:
        print(f"Retry attempt {attempt + 1} - Service Unavailable. Retrying in {retry_delay} seconds...")
        time.sleep(retry_delay)
    else:
        print(f"Error: {webpage.status_code}")
        break


Retry attempt 1 - Service Unavailable. Retrying in 5 seconds...
Retry attempt 2 - Service Unavailable. Retrying in 5 seconds...
Retry attempt 3 - Service Unavailable. Retrying in 5 seconds...
Retry attempt 4 - Service Unavailable. Retrying in 5 seconds...
Retry attempt 5 - Service Unavailable. Retrying in 5 seconds...
Retry attempt 6 - Service Unavailable. Retrying in 5 seconds...
Request successful!


In [57]:
# Title
def getTitle(soup):
    titleValue = ""
    try:
        title = soup.find("span",attrs={"id": "productTitle"})
        titleText = title.text
        titleValue = titleText.strip()
    except AttributeError:
        titleValue = ""
    
    return titleValue
        

In [58]:
def getPrice(soup):
    priceValue = ""
    try:
        price = soup.find("span",attrs={"class": "a-price-whole"})
        priceValue = price.text.strip()
    except AttributeError:
        priceValue = ""
    
    return priceValue

In [59]:
# Rating
def getRating(soup):    
    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	
    rating = rating[:3]
    return rating

In [61]:
if __name__ == '__main__':
    # Your existing code for making the initial request
    URL = 'https://www.amazon.in/s?k=playstation&crid=L285YXF77263&sprefix=playstati%2Caps%2C695&ref=nb_sb_ss_ts-doa-p_2_9'
    HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'}
    soup = BeautifulSoup(webpage.content, 'html.parser')
    # List of links (a - tag href)
    
    links = soup.find_all("a", attrs={'class':"a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})
    linksList = []
    for link in links:
            linksList.append(link.get('href'))
    
    dic = {'title':[], 'price':[], 'rating':[]}
    
    for link in linksList:
        if not link.startswith(('http://', 'https://')):
            link = 'https://www.amazon.in' + link
        newPage = requests.get(link, headers=HEADERS)
        newSoup = BeautifulSoup(newPage.content, 'html.parser')
        dic['title'].append(getTitle(newSoup))
        dic['price'].append(getPrice(newSoup))
        dic['rating'].append(getRating(newSoup))

                         
                         
    df = pd.DataFrame.from_dict(dic)
    df['title'].replace('',np.nan,inplace=True)
    df = df.dropna(subset=['title'])
    df.to_csv("EcommerceDataset.csv",header=True,index=False)